In [1]:
import os
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Breast_Tumor_Classification\\research'

In [2]:
os.chdir("../")
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Breast_Tumor_Classification'

In [3]:
#d715d4e3ea26e0aad712816503f3e9a25a4ebc46

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/krunalgedia/Breast_Tumor_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="krunalgedia"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d715d4e3ea26e0aad712816503f3e9a25a4ebc46"


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTrainingValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_dir: str
    benign_dir: str
    malignant_dir: str
    normal_dir: str
    benign: str
    normal: str
    malignant: str

In [5]:
from tumorClassify.constants import *
#from tumorClassify.utils.common import read_yaml, create_directories, compute_metrics, scale_bounding_box, create_bounding_box
#from tumorClassify.utils.common import DocumentClassificationDataset

In [6]:
from tumorClassify.utils.common import read_yaml, create_directories
from tumorClassify.utils.classification_utils import TrainValEval

C:\Users\kbged\Miniconda3\envs\tumormlflow\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\kbged\Miniconda3\envs\tumormlflow\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file a

In [7]:
read_yaml(CONFIG_FILE_PATH).data_training_validation.benign_dir

2024-07-14 14:02:54 | INFO | common.py:30 | yaml file: src\tumorClassify\config\config.yaml loaded successfully


'artifacts/data_ingestion/data/benign'

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_training_validation_config(self) -> DataTrainingValidationConfig:
        
        config = self.config.data_training_validation
        print(config)

        create_directories([config.root_dir])

        data_training_validation_config = DataTrainingValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_dir=config.unzip_dir,
            benign_dir=config.benign_dir,
            malignant_dir=config.malignant_dir,
            normal_dir=config.normal_dir,
            benign = config.benign,
            normal = config.normal,
            malignant = config.malignant,
            )

        return data_training_validation_config

In [9]:
import os
from tumorClassify import logger

In [10]:
config = ConfigurationManager()
data_training_validation_config = config.get_data_training_validation_config()

trainValEval = TrainValEval(config.config, config.params)

x_benign, y_benign, y_benign_label = trainValEval.load_data(data_training_validation_config.benign)
x_malignant, y_malignant, y_malignant_label = trainValEval.load_data(data_training_validation_config.malignant)
x_normal, y_normal, y_normal_label = trainValEval.load_data(data_training_validation_config.normal)

trainValEval.prepare_dataset(x_benign, x_malignant, x_normal, y_benign_label, y_malignant_label, y_normal_label)
trainValEval.prepare_model()


2024-07-14 14:02:54 | INFO | common.py:30 | yaml file: src\tumorClassify\config\config.yaml loaded successfully
2024-07-14 14:02:54 | INFO | common.py:30 | yaml file: params.yaml loaded successfully
2024-07-14 14:02:54 | INFO | common.py:53 | Directory already exists at: artifacts
{'root_dir': 'artifacts/data_training_validation', 'STATUS_FILE': 'artifacts/data_training_validation/status.txt', 'unzip_dir': 'artifacts/data_ingestion/data', 'benign_dir': 'artifacts/data_ingestion/data/benign', 'malignant_dir': 'artifacts/data_ingestion/data/malignant', 'normal_dir': 'artifacts/data_ingestion/data/normal', 'benign': 'benign', 'malignant': 'malignant', 'normal': 'normal'}
2024-07-14 14:02:54 | INFO | common.py:53 | Directory already exists at: artifacts/data_training_validation
64 tf.data.AUTOTUNE
1 tf.data.AUTOTUNE




In [11]:
trainValEval.train_model()



10/10 [==============================] - ETA: 0s - loss: 6.5008 - recall_c0: 0.3612 - recall_c1: 0.3389 - recall_c2: 0.3464 - precision_c0: 0.1816 - precision_c1: 0.5463 - precision_c2: 0.3002 
Epoch 1: val_loss improved from inf to 7.18100, saving model to VGG19weights.hdf5


C:\Users\kbged\Miniconda3\envs\tumormlflow\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 2s 2s/step
INFO:tensorflow:Assets written to: C:\Users\kbged\AppData\Local\Temp\tmpdwe2es7_\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\kbged\AppData\Local\Temp\tmpdwe2es7_\model\data\model\assets
2024/07/14 14:09:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\kbged\AppData\Local\Temp\tmpdwe2es7_\model, flavor: tensorflow). Fall back to return ['tensorflow==2.15.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
2024/07/14 14:09:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\kbged\Miniconda3\envs\tumormlflow\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml"
2024/07/14 14:10:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a sig

INFO:tensorflow:Assets written to: C:\Users\kbged\AppData\Local\Temp\tmpe7miu5ky\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\kbged\AppData\Local\Temp\tmpe7miu5ky\model\data\model\assets
2024/07/14 14:11:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\kbged\AppData\Local\Temp\tmpe7miu5ky\model, flavor: tensorflow). Fall back to return ['tensorflow==2.15.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
Successfully registered model 'VGG19'.
2024/07/14 14:12:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG19, version 1
Created version '1' of model 'VGG19'.


In [24]:
m = {'loss': [6.234276294708252, 5.808884620666504], 'recall_c0': [0.2055555284023285, 0.40940171480178833], 'recall_c1': [0.4064103364944458, 0.45662397146224976], 'recall_c2': [0.24871791899204254, 0.414743572473526], 'precision_c0': [0.1707264930009842, 0.2589743435382843], 'precision_c1': [0.6183761358261108, 0.7745727300643921], 'precision_c2': [0.2350427210330963, 0.3741452991962433], 'val_loss': [5.746504783630371, 5.556178569793701], 'val_recall_c0': [0.06410256028175354, 0.08974358439445496], 'val_recall_c1': [0.4358973801136017, 0.3205128014087677], 'val_recall_c2': [0.05769230052828789, 0.12179485708475113], 'val_precision_c0': [0.06410256028175354, 0.08974358439445496], 'val_precision_c1': [0.4358973801136017, 0.3205128014087677], 'val_precision_c2': [0.05769230052828789, 0.12179485708475113], 'lr': [0.0002, 0.0002]}
for k,v in m.items():
    print(k,v)
    #for key, value in logs.items():
    #    mlflow.log_metric(key, value, step=epoch)

loss [6.234276294708252, 5.808884620666504]
recall_c0 [0.2055555284023285, 0.40940171480178833]
recall_c1 [0.4064103364944458, 0.45662397146224976]
recall_c2 [0.24871791899204254, 0.414743572473526]
precision_c0 [0.1707264930009842, 0.2589743435382843]
precision_c1 [0.6183761358261108, 0.7745727300643921]
precision_c2 [0.2350427210330963, 0.3741452991962433]
val_loss [5.746504783630371, 5.556178569793701]
val_recall_c0 [0.06410256028175354, 0.08974358439445496]
val_recall_c1 [0.4358973801136017, 0.3205128014087677]
val_recall_c2 [0.05769230052828789, 0.12179485708475113]
val_precision_c0 [0.06410256028175354, 0.08974358439445496]
val_precision_c1 [0.4358973801136017, 0.3205128014087677]
val_precision_c2 [0.05769230052828789, 0.12179485708475113]
lr [0.0002, 0.0002]


In [25]:
# Log metrics for each epoch
metrics = m.keys()
for epoch in range(len(m[list(metrics)[0]])):
    for metric in metrics:
        print(metric, m[metric][epoch], epoch)
        #mlflow.log_metric(metric, history.history[metric][epoch], step=epoch)

loss 6.234276294708252 0
recall_c0 0.2055555284023285 0
recall_c1 0.4064103364944458 0
recall_c2 0.24871791899204254 0
precision_c0 0.1707264930009842 0
precision_c1 0.6183761358261108 0
precision_c2 0.2350427210330963 0
val_loss 5.746504783630371 0
val_recall_c0 0.06410256028175354 0
val_recall_c1 0.4358973801136017 0
val_recall_c2 0.05769230052828789 0
val_precision_c0 0.06410256028175354 0
val_precision_c1 0.4358973801136017 0
val_precision_c2 0.05769230052828789 0
lr 0.0002 0
loss 5.808884620666504 1
recall_c0 0.40940171480178833 1
recall_c1 0.45662397146224976 1
recall_c2 0.414743572473526 1
precision_c0 0.2589743435382843 1
precision_c1 0.7745727300643921 1
precision_c2 0.3741452991962433 1
val_loss 5.556178569793701 1
val_recall_c0 0.08974358439445496 1
val_recall_c1 0.3205128014087677 1
val_recall_c2 0.12179485708475113 1
val_precision_c0 0.08974358439445496 1
val_precision_c1 0.3205128014087677 1
val_precision_c2 0.12179485708475113 1
lr 0.0002 1


In [ ]:
# Log metrics for each epoch
for epoch, logs in enumerate(history.history):
    for key, value in logs.items():
        mlflow.log_metric(key, value, step=epoch)

In [30]:
config = ConfigurationManager()
data_training_validation_config = config.get_data_training_validation_config()


TrainValEval = TrainValEval(config.config, config.params)
TrainValEval.load_data(config.benign)

#model = 'ResNet152V2'
#model = 'VGG19'
#model = 'EfficientNetB7'
#model = 'EfficientNetV2S'
#model = 'ConvNeXtBase'
#model = 'vit_b16'

x_benign, y_benign, y_benign_label = TrainValEval.load_data(config.benign)
x_malignant, y_malignant, y_malignant_label = TrainValEval.load_data(config.malignant)
x_normal, y_normal, y_normal_label = TrainValEval.load_data(config.normal)

#load_data(config.DATA_DIR, config.SIZE, config.benign, model )
#x_malignant, y_malignant = load_data(config.DATA_DIR, config.SIZE, config.malignant, model)
#x_normal, y_normal = load_data(config.DATA_DIR, config.SIZE, config.normal, model)

if config.DEPTH==1:
  # prepare data to modeling
  x_benign = np.expand_dims(x_benign, -1)
  y_benign = np.expand_dims(y_benign, -1)

  # prepare data to modeling
  x_malignant = np.expand_dims(x_malignant, -1)
  y_malignant = np.expand_dims(y_malignant, -1)

  # prepare data to modeling
  x_normal = np.expand_dims(x_normal, -1)
  y_normal = np.expand_dims(y_normal, -1)

y_normal_label    = np.array([[1,0,0] for i in range(len(y_normal))])
y_benign_label    = np.array([[0,1,0] for i in range(len(y_benign))])
y_malignant_label = np.array([[0,0,1] for i in range(len(y_malignant))])

x_total_label = np.concatenate((x_benign, x_malignant, x_normal), axis=0)
y_total_label = np.concatenate((y_benign_label, y_malignant_label, y_normal_label), axis=0)
random_state = 24
y_total_label = np.expand_dims(y_total_label, -1)

X_train_total_label, X_test_total_label, y_train_total_label, y_test_total_label = train_test_split(
                                                                                   x_total_label,
                                                                                   y_total_label,
                                                                                   test_size=config.TEST_SIZE,
                                                                                   shuffle=config.SHUFFLE,
                                                                                   random_state=config.SEED)


#y_malignant = np.expand_dims(y_malignant, -1)
y_train_total_label = np.reshape(y_train_total_label,(-1,3))
y_test_total_label = np.reshape(y_test_total_label,(-1,3))

ny = len(y_benign_label) + len(y_malignant_label) + len(y_normal_label)
n_classes = 3
class_weight = {0: ny/(n_classes*len(y_normal_label)),
                1: ny/(n_classes*len(y_benign_label)),
                2: ny/(n_classes*len(y_malignant_label))}

ds_train_label = tf.data.Dataset.from_tensor_slices((X_train_total_label, y_train_total_label))
ds_test_label  = tf.data.Dataset.from_tensor_slices((X_test_total_label, y_test_total_label))


def prepare_dataset(x,y,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.batch(batch_size).cache().prefetch(buffer_size=config.AUTOTUNE)
    return dataset

dataset_train_label = prepare_dataset(X_train_total_label, y_train_total_label, config.BATCH_SIZE)
#dataset_val = prepare_dataset(val_df_scaled,62)
dataset_test_label = prepare_dataset(X_test_total_label,y_test_total_label,1)


POOLING = 'max'

# Input layer
input_layer = tf.keras.layers.Input(shape=(224, 224, 3))

if model=='ResNet152V2':
  pretrained_model = tf.keras.applications.ResNet152V2(
    include_top=False,
    input_shape=(config.SIZE, config.SIZE, config.DEPTH),
    weights="imagenet",
    pooling=POOLING,
    )(input_layer)
  pretrained_model.trainable=False
elif model=='VGG19':
  pretrained_model = tf.keras.applications.VGG19(
    include_top=False,
    input_shape=(config.SIZE, config.SIZE, config.DEPTH),
    pooling=POOLING,
    weights='imagenet'
    )(input_layer)
elif model=='EfficientNetB7':
  pretrained_model = tf.keras.applications.EfficientNetB7(
    include_top=False,
    input_shape=(config.SIZE, config.SIZE, config.DEPTH),
    weights="imagenet",
    pooling=POOLING,
    )(input_layer)
elif model=='EfficientNetV2S':
  pretrained_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    input_shape=(config.SIZE, config.SIZE, config.DEPTH),
    weights="imagenet",
    pooling=POOLING,
    )(input_layer)
elif model=='ConvNeXtBase':
  pretrained_model = tf.keras.applications.ConvNeXtBase(
    model_name="convnext_base",
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    pooling=POOLING,
    )(input_layer)
elif model=='vit_b16':
  pretrained_model = vit.vit_b16(
    #image_size=image_size,
    #activation='sigmoid',
    pretrained = True,
    include_top = False,
    pretrained_top = False,
    )(input_layer)

# Flatten layer
flattened = tf.keras.layers.Flatten()(pretrained_model)

# Dense layers
#dense1 = tf.keras.layers.Dense(256, activation='relu',
              #kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
              #bias_regularizer=regularizers.l2(0.01))(flattened)
#batch_norm1 = tf.keras.layers.BatchNormalization()(dense1)
#dropout1 = tf.keras.layers.Dropout(0.5)(batch_norm1)
dense1 = tf.keras.layers.Dense(256, activation='relu',
              kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
              bias_regularizer=regularizers.l2(0.01))(flattened)
batch_norm1 = tf.keras.layers.BatchNormalization()(dense1)
dropout1 = tf.keras.layers.Dropout(0.3)(batch_norm1)
dense2 = tf.keras.layers.Dense(64, activation='relu',
              kernel_regularizer=regularizers.l2(0.01),  # L2 regularization
              bias_regularizer=regularizers.l2(0.01),)(dropout1)
batch_norm2 = tf.keras.layers.BatchNormalization()(dense2)
dropout2 = tf.keras.layers.Dropout(0.2)(batch_norm2)
output_layer = tf.keras.layers.Dense(3, activation='softmax')(dropout2)


# Functional model
fine_tune_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
fine_tune_model.layers[1].trainable=False
pretrained_model.trainable=False

def recall(y_true, y_pred, c):
    pred_labels = K.cast(K.argmax(y_pred, axis=-1), K.floatx())
    true_labels = K.cast(K.argmax(y_true, axis=-1), K.floatx())

    tp = K.sum(K.cast(tf.logical_and(true_labels == c, pred_labels == c),K.floatx()))
    fn = K.sum(K.cast(tf.logical_and(true_labels == c, pred_labels != c),K.floatx()))
    tn = K.sum(K.cast(tf.logical_and(true_labels != c, pred_labels != c),K.floatx()))
    fp = K.sum(K.cast(tf.logical_and(true_labels != c, pred_labels == c),K.floatx()))

    sensitivity = (tp)/(tp+fn+0.0000001)
    return sensitivity


def precision(y_true, y_pred, c):
    pred_labels = K.cast(K.argmax(y_pred, axis=-1), K.floatx())
    true_labels = K.cast(K.argmax(y_true, axis=-1), K.floatx())

    tp = K.sum(K.cast(tf.logical_and(true_labels == c, pred_labels == c),K.floatx()))
    fn = K.sum(K.cast(tf.logical_and(true_labels == c, pred_labels != c),K.floatx()))
    tn = K.sum(K.cast(tf.logical_and(true_labels != c, pred_labels != c),K.floatx()))
    fp = K.sum(K.cast(tf.logical_and(true_labels != c, pred_labels == c),K.floatx()))

    precision = (tp)/(tp+fp+0.0000001)
    return precision

def recall_c0(y_true, y_pred):
    return recall(y_true, y_pred, 0)

def precision_c0(y_true, y_pred):
    return precision(y_true, y_pred, 0)

def recall_c1(y_true, y_pred):
    return recall(y_true, y_pred, 1)

def precision_c1(y_true, y_pred):
    return precision(y_true, y_pred, 1)

def recall_c2(y_true, y_pred):
    return recall(y_true, y_pred, 2)

def precision_c2(y_true, y_pred):
    return precision(y_true, y_pred, 2)



METRICS = [
      #tf.keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      #tf.keras.metrics.MeanSquaredError(name='Brier score'),
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      #tf.keras.metrics.Accuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

fine_tune_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
              #loss=dice_coef_loss,
              #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              #metrics = [metric_meaniou(),tf.keras.metrics.BinaryIoU(name='IoU')],
              metrics = [recall_c0,recall_c1,recall_c2,
                         precision_c0,precision_c1,precision_c2],
              #metrics = [recall_c2],
              #run_eagerly=True
              )

#config.NUM_EPOCHS=1
checkpoint = ModelCheckpoint(model+'weights.hdf5' ,
                             monitor = 'val_loss',
                             verbose = 1,
                             save_best_only=True,
                             mode = 'min',
                             save_weights_only=True,
                             save_freq='epoch'
                            )

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=15, min_lr=0.00005)

callbacks_list = [checkpoint, reduce_lr]

# Fit model
history = fine_tune_model.fit(dataset_train_label,
                    validation_data=dataset_test_label,
                    #steps_per_epoch=len(X_train)/(6),
                    #validation_steps=10,
                    callbacks=callbacks_list,
                    epochs=config.NUM_EPOCHS,
                    class_weight=class_weight,
                    #verbose=2,
                    )

2024-07-13 22:46:48 | INFO | common.py:30 | yaml file: src\tumorClassify\config\config.yaml loaded successfully
2024-07-13 22:46:48 | INFO | common.py:30 | yaml file: params.yaml loaded successfully
2024-07-13 22:46:48 | INFO | common.py:53 | Directory already exists at: artifacts
{'root_dir': 'artifacts/data_training_validation', 'STATUS_FILE': 'artifacts/data_training_validation/status.txt', 'unzip_dir': 'artifacts/data_ingestion/data', 'benign_dir': 'artifacts/data_ingestion/data/benign', 'malignant_dir': 'artifacts/data_ingestion/data/malignant', 'normal_dir': 'artifacts/data_ingestion/data/normal', 'benign': 'benign', 'malignant': 'malignant', 'normal': 'normal'}
2024-07-13 22:46:48 | INFO | common.py:53 | Directory already exists at: artifacts/data_training_validation


BoxKeyError: "'ConfigBox' object has no attribute 'unzip_dir'"

In [13]:
from tqdm.notebook import tqdm
import torch
import pandas as pd
import json
from PIL import Image
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader


class TrainAndValidate:
    def __init__(self, config: DataTrainingValidationConfig):
        self.feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
        self.tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base")
        self.processor = LayoutLMv3Processor(self.feature_extractor, self.tokenizer)
        self.config = config
        self.DOCUMENT_CLASSES = sorted(list(map(lambda p: p.name,Path(self.config.unzip_dir).glob("*"))))    

    def get_train_test_path(self):
        # Convert PosixPath objects to strings
        image_paths=sorted(list(Path(self.config.unzip_dir).glob("*/*.png")))
        image_paths_str = [str(path) for path in image_paths]
        
        # Define labels based on whether the paths contain specific strings
        income_labels = ["income" in path for path in image_paths_str]
        balance_labels = ["balance" in path for path in image_paths_str]
        cashflow_labels = ["cashflow" in path for path in image_paths_str]
        
        # Use any one of the labels as the target for stratified split
        # Here, I'm using income_labels, but you can choose based on your requirements
        train_images_str, test_images_str = train_test_split(image_paths_str, test_size=0.2, stratify=income_labels, random_state=42)
        
        # Convert back to PosixPath objects
        train_images = [Path(path) for path in train_images_str]
        test_images = [Path(path) for path in test_images_str]

        return train_images, test_images
    
    def train(self, train_images, test_images):
        train_dataset = DocumentClassificationDataset(train_images, self.processor)
        valid_dataset = DocumentClassificationDataset(test_images, self.processor)
        
        train_dataloader = DataLoader(
            train_dataset,
            batch_size=1,
            shuffle=True,
            #num_workers=10
        )
        
        valid_dataloader = DataLoader(
            valid_dataset,
            batch_size=1,
            shuffle=False,
            #num_workers=10
        )

        device = "cuda:0" if torch.cuda.is_available() else "cpu"

        n_classes = len(self.DOCUMENT_CLASSES)
        
        model = LayoutLMv3ForSequenceClassification.from_pretrained(
                    "microsoft/layoutlmv3-base",
                    num_labels=n_classes
                )
        model.to(device)

        # load seqeval metric
        #metric = evaluate.load("seqeval")
        model.config.id2label = {k: v for k, v in enumerate(self.DOCUMENT_CLASSES)}
        model.config.label2id = {v: k for k, v in enumerate(self.DOCUMENT_CLASSES)}
        # labels of the model
        ner_labels = list(model.config.id2label.values())
        
        num_epochs = 1
        optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
        
        # Initialize an empty DataFrame to store the metrics
        columns = ["Epoch", "Training Loss", "Validation Loss", "Precision", "Recall", "F1", "Accuracy"]
        df_metrics = pd.DataFrame(columns=columns)
        
        # Early stopping parameters
        patience = 3 # Number of epochs to wait for improvement
        best_validation_loss = float('inf')
        current_patience = 0
        
        for epoch in range(num_epochs):
            print("Epoch:", epoch)
        
            # Training
            model.train()
            training_loss = 0.0
            num = 0
            for batch in tqdm(train_dataloader):
                labels = torch.Tensor(batch["labels"]).unsqueeze_(0).long().to(device)
                outputs = model(
                    input_ids=batch["input_ids"].to(device),
                    attention_mask=torch.tensor(batch["attention_mask"]).to(device),
                    bbox=torch.tensor(batch["bbox"]).to(device),
                    pixel_values=torch.tensor(batch["pixel_values"]).to(device),
                    labels=batch["labels"].to(device)
                )
                loss = outputs.loss
                training_loss += loss.item()
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                num += 1
        
            print("Training Loss:", training_loss / num)
        
            # Validation
            model.eval()
            preds = []
            labs = []
            validation_loss = 0.0
            num = 0
            for batch in tqdm(valid_dataloader):
                labels = torch.Tensor(batch["labels"]).to(device)
                outputs = model(
                    input_ids=batch["input_ids"].to(device),
                    attention_mask=torch.tensor(batch["attention_mask"]).to(device),
                    bbox=torch.tensor(batch["bbox"]).to(device),
                    pixel_values=torch.tensor(batch["pixel_values"]).to(device),
                    labels=labels
                )
                loss = outputs.loss
                preds_idx = outputs.logits.argmax(axis=1)
                labs.append(labels.tolist())
                preds.append(preds_idx.tolist())
                validation_loss += loss.item()
                num += 1
        
            print("Validation Loss:", validation_loss / num)
            print(preds)
            print(labs)
        
            overall_precision, overall_recall, overall_f1, overall_accuracy = compute_metrics([preds, labs])
            print("Overall Precision:", overall_precision)
            print("Overall Recall:", overall_recall)
        
            # Store metrics in the DataFrame
            metrics_data = {
                "Epoch": epoch,
                "Training Loss": training_loss,
                "Validation Loss": validation_loss,
                "Precision": overall_precision,
                "Recall": overall_recall,
                "F1": overall_f1,
                "Accuracy": overall_accuracy
            }
            #df_metrics = df_metrics.append(metrics_data, ignore_index=True)
            df_metrics.loc[len(df_metrics)] = metrics_data
        
            # Early stopping check
            if validation_loss < best_validation_loss:
                best_validation_loss = validation_loss
                current_patience = 0
            else:
                current_patience += 1
                if current_patience >= patience:
                    print(f"Early stopping! No improvement in validation loss for {patience} consecutive epochs.")
                    break
        
        # Save the DataFrame to a CSV file or do any further analysis
        df_metrics.to_csv("metrics.csv", index=False)
        print(df_metrics)

        return df_metrics
        
        # Convert DataFrame to markdown
        #markdown_table = df_metrics.to_markdown()
        
        # Print the markdown table
        #print(markdown_table)


        


In [14]:
try:
    config = ConfigurationManager()
    data_training_validation_config = config.get_data_training_validation_config()
    #data_training_validation = DataTrainingValidationConfig(config=data_training_validation_config)
    train_and_validate = TrainAndValidate(data_training_validation_config)
    train_images, test_images = train_and_validate.get_train_test_path()
    df = train_and_validate.train(train_images, test_images)
    #data_training_validation.prepare_all_files()
except Exception as e:
    raise e

[2024-02-06 12:03:27,659: INFO: common: yaml file: src\docClassify\config\config.yaml loaded successfully]
[2024-02-06 12:03:27,665: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-06 12:03:27,668: INFO: common: created directory at: artifacts]
[2024-02-06 12:03:27,672: INFO: common: created directory at: artifacts/data_training_validation]


C:\Users\kbged\Miniconda3\envs\doc_classify_aws\Lib\site-packages\transformers\models\layoutlmv3\feature_extraction_layoutlmv3.py:30: FutureWarning: The class LayoutLMv3FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv3ImageProcessor instead.
  warnings.warn(
Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0


  0%|          | 0/40 [00:00<?, ?it/s]

C:\Users\kbged\AppData\Local\Temp\ipykernel_3828\2638552435.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=torch.tensor(batch["attention_mask"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_3828\2638552435.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bbox=torch.tensor(batch["bbox"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_3828\2638552435.py:100: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pixel_values=torch.tensor(batch["pixel_values"]).to(device),
C:\Users\kbged\Miniconda3\envs\do

Training Loss: 1.1226830914616586


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\kbged\AppData\Local\Temp\ipykernel_3828\2638552435.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=torch.tensor(batch["attention_mask"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_3828\2638552435.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bbox=torch.tensor(batch["bbox"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_3828\2638552435.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pixel_values=torch.tensor(batch["pixel_values"]).to(device),


Validation Loss: 1.0879846274852754
[[0], [0], [2], [2], [2], [2], [2], [0], [2], [2]]
[[0], [1], [2], [0], [0], [2], [2], [1], [1], [0]]
Overall Precision: 0.26190476190476186
Overall Recall: 0.4
   Epoch  Training Loss  Validation Loss  Precision  Recall        F1  \
0      0      44.907324        10.879846   0.261905     0.4  0.294286   

   Accuracy  
0       0.4  


C:\Users\kbged\Miniconda3\envs\doc_classify_aws\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
test_images
sorted(list(map(lambda p: p.name,test_images[0].parent.parent.glob("*"))))

['balance sheet', 'cashflow', 'income statement']

In [13]:
feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base")
processor = LayoutLMv3Processor(feature_extractor, tokenizer)

C:\Users\kbged\Miniconda3\envs\doc_classify_aws\Lib\site-packages\transformers\models\layoutlmv3\feature_extraction_layoutlmv3.py:30: FutureWarning: The class LayoutLMv3FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv3ImageProcessor instead.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

C:\Users\kbged\Miniconda3\envs\doc_classify_aws\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kbged\.cache\huggingface\hub\models--microsoft--layoutlmv3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from pathlib import Path

# Convert PosixPath objects to strings
image_paths=sorted(list(Path("artifacts/data_ingestion/data/").glob("*/*.png")))
image_paths_str = [str(path) for path in image_paths]

# Define labels based on whether the paths contain specific strings
income_labels = ["income" in path for path in image_paths_str]
balance_labels = ["balance" in path for path in image_paths_str]
cashflow_labels = ["cashflow" in path for path in image_paths_str]

# Use any one of the labels as the target for stratified split
# Here, I'm using income_labels, but you can choose based on your requirements
train_images_str, test_images_str = train_test_split(image_paths_str, test_size=0.2, stratify=income_labels, random_state=42)

# Convert back to PosixPath objects
train_images = [Path(path) for path in train_images_str]
test_images = [Path(path) for path in test_images_str]

DOCUMENT_CLASSES = sorted(list(map(
    lambda p: p.name,
    Path("artifacts/data_ingestion/data/").glob("*")
)))
DOCUMENT_CLASSES
test_images

[WindowsPath('artifacts/data_ingestion/data/balance sheet/bs7.png'),
 WindowsPath('artifacts/data_ingestion/data/cashflow/cf9.png'),
 WindowsPath('artifacts/data_ingestion/data/income statement/is4.png'),
 WindowsPath('artifacts/data_ingestion/data/balance sheet/bs16.png'),
 WindowsPath('artifacts/data_ingestion/data/balance sheet/bs15.png'),
 WindowsPath('artifacts/data_ingestion/data/income statement/is8.png'),
 WindowsPath('artifacts/data_ingestion/data/income statement/is2.png'),
 WindowsPath('artifacts/data_ingestion/data/cashflow/cf5.png'),
 WindowsPath('artifacts/data_ingestion/data/cashflow/cf12.png'),
 WindowsPath('artifacts/data_ingestion/data/balance sheet/bs3.png')]

In [26]:
train_dataset = DocumentClassificationDataset(train_images, processor)
valid_dataset = DocumentClassificationDataset(test_images, processor)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    #num_workers=10
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    #num_workers=10
)

In [30]:
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification

device = "cuda:0" if torch.cuda.is_available() else "cpu"

n_classes = len(DOCUMENT_CLASSES)

model = LayoutLMv3ForSequenceClassification.from_pretrained(
            "microsoft/layoutlmv3-base",
            num_labels=n_classes
        )
model.to(device)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMv3ForSequenceClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Enco

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tqdm.notebook import tqdm
import torch
import pandas as pd
import json
from PIL import Image


# load seqeval metric
#metric = evaluate.load("seqeval")
model.config.id2label = {k: v for k, v in enumerate(DOCUMENT_CLASSES)}
model.config.label2id = {v: k for k, v in enumerate(DOCUMENT_CLASSES)}
# labels of the model
ner_labels = list(model.config.id2label.values())

num_epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)

# Initialize an empty DataFrame to store the metrics
columns = ["Epoch", "Training Loss", "Validation Loss", "Precision", "Recall", "F1", "Accuracy"]
df_metrics = pd.DataFrame(columns=columns)

# Early stopping parameters
patience = 3 # Number of epochs to wait for improvement
best_validation_loss = float('inf')
current_patience = 0

for epoch in range(num_epochs):
    print("Epoch:", epoch)

    # Training
    model.train()
    training_loss = 0.0
    num = 0
    for batch in tqdm(train_dataloader):
        labels = torch.Tensor(batch["labels"]).unsqueeze_(0).long().to(device)
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=torch.tensor(batch["attention_mask"]).to(device),
            bbox=torch.tensor(batch["bbox"]).to(device),
            pixel_values=torch.tensor(batch["pixel_values"]).to(device),
            labels=batch["labels"].to(device)
        )
        loss = outputs.loss
        training_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        num += 1

    print("Training Loss:", training_loss / num)

    # Validation
    model.eval()
    preds = []
    labs = []
    validation_loss = 0.0
    num = 0
    for batch in tqdm(valid_dataloader):
        labels = torch.Tensor(batch["labels"]).to(device)
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=torch.tensor(batch["attention_mask"]).to(device),
            bbox=torch.tensor(batch["bbox"]).to(device),
            pixel_values=torch.tensor(batch["pixel_values"]).to(device),
            labels=labels
        )
        loss = outputs.loss
        preds_idx = outputs.logits.argmax(axis=1)
        labs.append(labels.tolist())
        preds.append(preds_idx.tolist())
        validation_loss += loss.item()
        num += 1

    print("Validation Loss:", validation_loss / num)
    print(preds)
    print(labs)

    overall_precision, overall_recall, overall_f1, overall_accuracy = compute_metrics([preds, labs])
    print("Overall Precision:", overall_precision)
    print("Overall Recall:", overall_recall)

    # Store metrics in the DataFrame
    metrics_data = {
        "Epoch": epoch,
        "Training Loss": training_loss,
        "Validation Loss": validation_loss,
        "Precision": overall_precision,
        "Recall": overall_recall,
        "F1": overall_f1,
        "Accuracy": overall_accuracy
    }
    df_metrics = df_metrics.append(metrics_data, ignore_index=True)

    # Early stopping check
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        current_patience = 0
    else:
        current_patience += 1
        if current_patience >= patience:
            print(f"Early stopping! No improvement in validation loss for {patience} consecutive epochs.")
            break

# Save the DataFrame to a CSV file or do any further analysis
df_metrics.to_csv("metrics.csv", index=False)
df_metrics

# Convert DataFrame to markdown
markdown_table = df_metrics.to_markdown()

# Print the markdown table
print(markdown_table)

Epoch: 0


  0%|          | 0/40 [00:00<?, ?it/s]

C:\Users\kbged\AppData\Local\Temp\ipykernel_51808\124497835.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=torch.tensor(batch["attention_mask"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_51808\124497835.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bbox=torch.tensor(batch["bbox"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_51808\124497835.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pixel_values=torch.tensor(batch["pixel_values"]).to(device),
C:\Users\kbged\Miniconda3\envs\doc

Training Loss: 1.086370076239109


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\kbged\AppData\Local\Temp\ipykernel_51808\124497835.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=torch.tensor(batch["attention_mask"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_51808\124497835.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bbox=torch.tensor(batch["bbox"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_51808\124497835.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pixel_values=torch.tensor(batch["pixel_values"]).to(device),


Validation Loss: 1.083499014377594
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
[[0], [1], [2], [0], [0], [2], [2], [1], [1], [0]]


NameError: name 'compute_metrics' is not defined